In [1]:
import pandas as pd
import numpy as np

# 1164, 129
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/1164-train-from-1293-spmm-over-3s.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/129-test-from-1293-spmm-over-3s.csv')

# 2268, 226
# train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/2042-train-from-2268-spmm-over-3s.csv')
# test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/226-test-from-2268-spmm-over-3s.csv')

# 1727, 191
# train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/1727-train-from-1918-spmm-over-3s.csv')
# test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/191-test-from-1918-spmm-over-3s.csv')


In [2]:
# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd']] 
y_train = train['bz_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd']] 
y_test = test['bz_smdm']

In [3]:
from sklearn.metrics import r2_score, mean_squared_error

# Metric
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def custom_scoring(real, pred):
    rmse = np.sqrt(mean_squared_error(real, pred))
    r2 = r2_score(real, pred)
    mape = mean_absolute_percentage_error(real, pred)
    return mape

In [4]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

# 탐색 대상 함수 (XGBRegressor)
def XGB(
max_depth,
learning_rate, 
n_estimators, 
min_child_weight, 
subsample,
_lambda,
#gamma ,
#colsample_bytree, 
#_alpha,
silent=True, 
n_jobs=-1):
    
    # 모델 정의
    model = xgb.XGBRegressor( 
objective = 'reg:squarederror',
max_depth=int(max_depth),
learning_rate=learning_rate,
n_estimators=int(n_estimators),
min_child_weight=min_child_weight,
subsample=subsample,
reg_lambda=_lambda,        
#gamma=gamma,
#colsample_bytree=colsample_bytree, 
#reg_alpha=_alpha
n_jobs=n_jobs        
                              )
    
    # bayesian optimization을 통해 파라미터를 받아
    # Train을 Train + Validation으로 나눠 cross-validation 성능 확인
    kfold = KFold(n_splits=9, shuffle = True, random_state=0)

    # cross-validation 평균 성능 성능 확인
    score = cross_val_score(model,
                            X_train, 
                            y_train, 
                            cv=kfold,
                            scoring=make_scorer(custom_scoring,greater_is_better=False),
                            n_jobs=-1
                           ).mean()
    
    return score

In [5]:
from bayes_opt import BayesianOptimization

# # 하이퍼파라미터 정의
# pbounds = {
# 'max_depth': (12, 16), 
# 'learning_rate': (0.01, 0.3),
# 'n_estimators': (50, 100),
# 'min_child_weight': (0.1, 1),
# 'subsample': (0.5, 1), 
# '_lambda' : (0.1,1),    
# #'gamma': (0, 0.3),     
# #'colsample_bytree' :(0.5, 1)
# #'_alpha' : (0,1)           
#                       }

# 하이퍼파라미터 정의
pbounds = {
'max_depth': (12, 16), 
'learning_rate': (0.01, 0.2),
'n_estimators': (50, 150),
'min_child_weight': (0.1, 1),
'subsample': (0.1, 1), 
'_lambda' : (0.1,1),    
#'gamma': (0, 0.3),     
#'colsample_bytree' :(0.5, 1)
#'_alpha' : (0,1)           
                      }

# Bayesian optimization 객체 생성
bo=BayesianOptimization(f=XGB, pbounds=pbounds, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행 (파라미터 넣고 목적함수 값 출력하고)
bo.maximize(init_points=2, n_iter=50, acq='ei', xi=0.01)

# 뽑힌 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

|   iter    |  target   |  _lambda  | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -46.46    |  0.4753   |  0.1469   |  12.0     |  0.3721   |  64.68    |  0.1831   |
|  2        | -15.91    |  0.2676   |  0.07566  |  13.59    |  0.5849   |  91.92    |  0.7167   |
|  3        | -16.29    |  0.2233   |  0.07657  |  13.92    |  0.4265   |  91.26    |  0.6252   |
|  4        | -38.18    |  0.1      |  0.01     |  16.0     |  0.1      |  107.0    |  0.1      |
|  5        | -17.89    |  1.0      |  0.2      |  12.0     |  1.0      |  90.49    |  1.0      |
|  6        | -17.89    |  1.0      |  0.2      |  12.0     |  1.0      |  150.0    |  1.0      |
|  7        | -35.54    |  0.1      |  0.01     |  16.0     |  0.1      |  140.5    |  0.1      |
|  8        | -17.74    |  1.0      |  0.2      |  16.0     |  1.0      |  93.6     |  1.0      |
|  9        | -23.02

|  51       | -17.78    |  1.0      |  0.2      |  13.11    |  0.8673   |  126.9    |  1.0      |
|  52       | -19.87    |  0.1      |  0.2      |  12.03    |  1.0      |  127.0    |  1.0      |
{'target': -15.91454911418465, 'params': {'_lambda': 0.2676341902399038, 'learning_rate': 0.07565653813817906, 'max_depth': 13.58706989692268, 'min_child_weight': 0.5849350606030213, 'n_estimators': 91.91945144032948, 'subsample': 0.7166975503570835}}



In [6]:
import xgboost as xgb

# Train + Valid cross-validation을 거친, 최적의 하이퍼파라미터를 사용
best_model = xgb.XGBRegressor(
objective = 'reg:squarederror',
max_depth=int(bo.max['params']['max_depth']),
learning_rate=bo.max['params']['learning_rate'],
n_estimators=int(bo.max['params']['n_estimators']),
min_child_weight=bo.max['params']['min_child_weight'],
subsample=bo.max['params']['subsample'],
reg_lambda = bo.max['params']['_lambda'],    
#gamma=bo.max['params']['gamma'],
#colsample_bytree=bo.max['params']['colsample_bytree'],
#reg_alpha = bo.max['params']['_alpha']
n_jobs=-1
                             )
# 모델 훈련
best_model.fit(X_train, y_train)

XGBRegressor(learning_rate=0.07565653813817906, max_depth=13,
             min_child_weight=0.5849350606030213, n_estimators=91, n_jobs=-1,
             objective='reg:squarederror', reg_lambda=0.2676341902399038,
             subsample=0.7166975503570835)

In [7]:
# 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

# 훈련데이터 예측
y_train_pred = best_model.predict(X_train)
print("-------- 훈련데이터 예측 --------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_train, y_train_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_train, y_train_pred)))
print("\n")

# 검증데이터 예측
print("-------- 검증데이터 예측 --------------------------")
print("mape : {}%".format(-bo.max['target']))
print("\n")

# 테스트데이터 예측
y_pred = best_model.predict(X_test)
print("-------- 테스트데이터 예측 -------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_test, y_pred)))
print("\n")

{'target': -15.91454911418465, 'params': {'_lambda': 0.2676341902399038, 'learning_rate': 0.07565653813817906, 'max_depth': 13.58706989692268, 'min_child_weight': 0.5849350606030213, 'n_estimators': 91.91945144032948, 'subsample': 0.7166975503570835}}

-------- 훈련데이터 예측 --------------------------
rmse : 4721.518805080943
mape : 1.2080575219233052%


-------- 검증데이터 예측 --------------------------
mape : 15.91454911418465%


-------- 테스트데이터 예측 -------------------------
rmse : 44143.070819499604
mape : 11.312353873325048%




In [8]:
# mape_list = {}
# # 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
# for idx,value in enumerate(y_test):
#     mape_temp = {}
#     predicate = int(y_pred[idx])
#     mape = abs((value - predicate) / value) * 100
#     mape_temp['pred'] = predicate
#     mape_temp['real'] = value
#     mape_temp['mape'] = mape
#     mape_list[idx] = mape_temp
# mape_list_sort = sorted(mape_list.values(), key=lambda x:(x['mape']), reverse=True)
# mape_list_sort  